In [1]:
import pandas as pd

# cargar DataSet con los tweets de entrenamiento
train_mx = pd.read_csv('mx-train-data-non-contextual.csv', names=['texto'],  header=0)# , nrows=30, header=1) 
# cargar las clases de cada tweet de entrenamiento
sol = pd.read_csv('mx-train-outputs.sol', names=["clase"])

train_mx.shape, sol.shape, train_mx.head(10), sol.head(10)

((5060, 1), (5060, 1),                                                texto
 0  Soy el Clint Eastwood de los Puentes de Madiso...
 1  Actualmente ya pasó de moda la pucha joto, aho...
 2  ¿Es cierto esto? Y no me refiero a lo que dijo...
 3  Vuela pega y esquiva... la neta está de la ver...
 4  Mejor puto disfraz de la noche!!!! 👊👊👊Por terc...
 5  YA SE CONVIRTIÓ LA K  EN VIRGEN MARIA ,AHORA S...
 6  Hoy en día ya casi nadie se muere por VIH, se ...
 7  Me quedé sin novia, sin amante &amp; sin ex .....
 8  Woou rulo invita..yo tambien quiero mamar esa ...
 9  Iba a escribir un tuit disculpándome por todos...,    clase
 0      0
 1      1
 2      0
 3      0
 4      0
 5      1
 6      1
 7      0
 8      1
 9      0)

In [2]:
import sklearn.model_selection as model_selection

# split del datataset completo en entrenamiento | test
X_train, X_test, y_train, y_test = model_selection.train_test_split(train_mx, sol, train_size=0.8, random_state=True)

# Juntar texto|clase y re-ordenar aleatoriamente 
X_y_train_shuffled = X_train.join(y_train).sample(frac=1)

X_train.shape, X_y_train_shuffled.shape, X_y_train_shuffled.head(5), X_y_train_shuffled.loc[[1]]

((4048, 1),
 (4048, 2),
                                                   texto  clase
 1496  DESDE LAS 7 HASTA LAS PUTAS 2 DE LA TARDE SIN ...      0
 801           Si te apasiona, rómpete la madre.#berraco      0
 2826  Si tu día va de la mierda, recuerda que hay pe...      0
 2677  PUTAS ENCUESTAS DE INSTAGRAM, SÓLO VINIERON A ...      0
 4212  Sí está de la verga que digan verga niñas, no ...      0,
                                                texto  clase
 1  Actualmente ya pasó de moda la pucha joto, aho...      1)

In [3]:
with pd.option_context('display.max_colwidth', -1):
    print(X_y_train_shuffled.loc[[154]]['texto'])
    print(sol.loc[[154]])

154    #YoNoPuedoEvitarQue insulte a los que no ponen sus putas direccionales.
Name: texto, dtype: object
     clase
154  0    


In [4]:
# aumentado de X_train / y_train con bi-frases
_ = X_y_train_shuffled.iloc[0]
train_bi_frase = pd.DataFrame([_], columns = ['texto', 'clase'])
train_bi_frase

for index, row in X_y_train_shuffled.iloc[1:].iterrows():
    try:
        train_bi_frase =  train_bi_frase.append(row)
        if _['clase'] == row['clase']:
            bi_frase = {'texto': '> ' + _['texto'] + ' | ' + row['texto'], 'clase':row['clase']}
            train_bi_frase = train_bi_frase.append(bi_frase, ignore_index=True)
            _ =  {'texto': '_VACIA_', 'clase': 9}
        else:   
            _ = row
    except Exception as exc:
        print(exc)
        _ = row
        continue
        
train_bi_frase.shape

(5517, 2)

In [5]:
# solo para checar algunas bi-frases
print(train_bi_frase.loc[(train_bi_frase['clase'] == 0)].head(13))
print(train_bi_frase.loc[(train_bi_frase['clase'] == 1)].head(13))

                                                texto  clase
0   DESDE LAS 7 HASTA LAS PUTAS 2 DE LA TARDE SIN ...      0
1           Si te apasiona, rómpete la madre.#berraco      0
2   > DESDE LAS 7 HASTA LAS PUTAS 2 DE LA TARDE SI...      0
3   Si tu día va de la mierda, recuerda que hay pe...      0
4   PUTAS ENCUESTAS DE INSTAGRAM, SÓLO VINIERON A ...      0
5   > Si tu día va de la mierda, recuerda que hay ...      0
6   Sí está de la verga que digan verga niñas, no ...      0
7   Me mama como me vale verga que se que tengo qu...      0
8   > Sí está de la verga que digan verga niñas, n...      0
9   Tas viendo que la gorda es aventada y le pones...      0
11  Está cagado que ayer no tomé alcohol y amanecí...      0
12  Gorda chancho me dijeron por Sarahah JAJAJAJAJ...      0
13  > Está cagado que ayer no tomé alcohol y amane...      0
                                                texto  clase
10  Pero si hubiera sido otro que si lo hubiera pa...      1
26  🇦🇷🎶😮🎶 ¡Así cantan en

In [6]:
# ver una bifrase completa por índice
with pd.option_context('display.max_colwidth', -1):
    print(train_bi_frase.loc[[2]]['texto'])
    print(train_bi_frase.loc[[28]]['texto'])

2    > DESDE LAS 7 HASTA LAS PUTAS 2 DE LA TARDE SIN HACER NADA, ME QUIERO COLGAAAAAAAR. | Si te apasiona, rómpete la madre.#berraco
Name: texto, dtype: object
28    > 🇦🇷🎶😮🎶 ¡Así cantan en el vestuario de #Argentina!  "No me importa lo que digan esos putos periodistas" | Tan de la verga está la selección Argentina que hoy debutó Darío Benedetto.
Name: texto, dtype: object


In [7]:
# re-split del train_bi_frase

X_train = train_bi_frase['texto']
y_train = train_bi_frase['clase']

X_train = X_train.squeeze() 
X_test = X_test.squeeze() 
y_train = y_train.squeeze() 
y_test = y_test.squeeze()

X_train.shape, y_train.shape, X_train.head(), y_train.head()

((5517,), (5517,), 0    DESDE LAS 7 HASTA LAS PUTAS 2 DE LA TARDE SIN ...
 1            Si te apasiona, rómpete la madre.#berraco
 2    > DESDE LAS 7 HASTA LAS PUTAS 2 DE LA TARDE SI...
 3    Si tu día va de la mierda, recuerda que hay pe...
 4    PUTAS ENCUESTAS DE INSTAGRAM, SÓLO VINIERON A ...
 Name: texto, dtype: object, 0    0
 1    0
 2    0
 3    0
 4    0
 Name: clase, dtype: int64)

In [8]:
# modelos para EvomSA

from EvoMSA.utils import download
from EvoMSA.base import EvoMSA

from sklearn import metrics
from sklearn.metrics import classification_report

haha = download('haha2018_Es.evomsa')
mexa3t = download('mexa3t2018_aggress_Es.evomsa')
misoginia = download('misoginia_Es.evomsa')

haha

'C:\\Users\\Calderas\\Anaconda3\\lib\\site-packages\\EvoMSA\\models\\haha2018_Es.evomsa'

In [9]:
# Clasificador con Modelos
evo = EvoMSA(TR=True, B4MSA=False, lang='es', Aggress=True,
                 stacked_method='sklearn.naive_bayes.GaussianNB',
                 models=[
                     ["text_model_inv.TextModelInv","sklearn.svm.LinearSVC"],
                     [misoginia, "sklearn.svm.LinearSVC"],
                     [haha, "sklearn.svm.LinearSVC"] ,
                     [mexa3t, "sklearn.svm.LinearSVC"]
                 ])

import time
start = time.time()
# Entrenamiento X_train | y_train
evo.fit(X_train, y_train)    
print(time.time() - start, "seconds")

# Predicción con X_test
pred = evo.predict(X_test)
print(pred)

C:\Users\Calderas\Anaconda3\lib\site-packages\sklearn\base.py:315: UserWarning: Trying to unpickle estimator LinearSVC from version 0.22.1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\Calderas\Anaconda3\lib\site-packages\sklearn\base.py:315: UserWarning: Trying to unpickle estimator GaussianNB from version 0.22.1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
100%|██████████| 3/3 [00:36<00:00, 12.21s/it]

100%|██████████| 1/1 [00:13<00:00, 13.46s/it]

100%|██████████| 4/4 [00:25<00:00,  6.48s/it]

100%|██████████| 5/5 [00:00<00:00, 124.89it/s]


165.81858587265015 seconds


100%|██████████| 3/3 [00:04<00:00,  1.57s/it]

100%|██████████| 1/1 [00:02<00:00,  2.14s/it]

100%|██████████| 4/4 [00:04<00:00,  1.09s/it]

100%|██████████| 6/6 [00:00<00:00, 602.25it/s]


[0 0 1 ... 0 0 1]


In [15]:
print("       f1 score micro:", metrics.f1_score(y_test, pred, average="micro"))
print("precision score micro:", metrics.precision_score(y_test, pred, average="micro"))
print("   recall score micro:", metrics.recall_score(y_test, pred, average="micro"))  

       f1 score micro: 0.8171936758893281
precision score micro: 0.817193675889328
   recall score micro: 0.817193675889328


In [17]:
## Con TextModelInv
print("       f1 score micro:", metrics.f1_score(y_test, pred, average="micro"))
print("precision score micro:", metrics.precision_score(y_test, pred, average="micro"))
print("   recall score micro:", metrics.recall_score(y_test, pred, average="micro"))  

       f1 score micro: 0.8231225296442687
precision score micro: 0.8231225296442688
   recall score micro: 0.8231225296442688


In [12]:
## Con Bi-Frase
print("       f1 score micro:", metrics.f1_score(y_test, pred, average="micro"))
print("precision score micro:", metrics.precision_score(y_test, pred, average="micro"))
print("   recall score micro:", metrics.recall_score(y_test, pred, average="micro"))  

       f1 score micro: 0.7885375494071146
precision score micro: 0.7885375494071146
   recall score micro: 0.7885375494071146


In [46]:
## Con Bi-Frase SIN IS-BI-FRASE
print("       f1 score macro:", metrics.f1_score(y_test, pred, average="macro"))
print("precision score macro:", metrics.precision_score(y_test, pred, average="macro"))
print("   recall score macro:", metrics.recall_score(y_test, pred, average="macro"))  
print("       f1 score micro:", metrics.f1_score(y_test, pred, average="micro"))
print("precision score micro:", metrics.precision_score(y_test, pred, average="micro"))
print("   recall score micro:", metrics.recall_score(y_test, pred, average="micro"))  

       f1 score macro: 0.7584124245038827
precision score macro: 0.796875
   recall score macro: 0.738244514106583
       f1 score micro: 0.825
precision score micro: 0.825
   recall score micro: 0.825


In [10]:
## Con Bi-Frase SIN IS-BI-FRASE
print("       f1 score macro:", metrics.f1_score(y_test, pred, average="macro"))
print("precision score macro:", metrics.precision_score(y_test, pred, average="macro"))
print("   recall score macro:", metrics.recall_score(y_test, pred, average="macro"))  
print("       f1 score micro:", metrics.f1_score(y_test, pred, average="micro"))
print("precision score micro:", metrics.precision_score(y_test, pred, average="micro"))
print("   recall score micro:", metrics.recall_score(y_test, pred, average="micro"))  

       f1 score macro: 0.7455019894328498
precision score macro: 0.7384106759332996
   recall score macro: 0.7545803592135439
       f1 score micro: 0.8003952569169961
precision score micro: 0.8003952569169961
   recall score micro: 0.8003952569169961


In [12]:
# Métricas contra y_test
print("F1_Score Macro: ", metrics.f1_score(y_test, pred, average="macro"))

F1_Score Macro:  0.757215014232268


In [13]:
target_names = ['Ofensivo', 'No-Ofensivo']
print(classification_report(y_test, pred, target_names=target_names, digits=4)) 

              precision    recall  f1-score   support

    Ofensivo     0.8750    0.8808    0.8779       755
 No-Ofensivo     0.6429    0.6304    0.6365       257

    accuracy                         0.8172      1012
   macro avg     0.7589    0.7556    0.7572      1012
weighted avg     0.8160    0.8172    0.8166      1012



In [14]:
metrics.precision_recall_fscore_support(y_test, pred, average="micro")

(0.817193675889328, 0.817193675889328, 0.8171936758893281, None)

In [15]:
print("       f1 score micro:", metrics.f1_score(y_test, pred, average="micro"))
print("precision score micro:", metrics.precision_score(y_test, pred, average="micro"))
print("   recall score micro:", metrics.recall_score(y_test, pred, average="micro"))    

       f1 score micro: 0.8171936758893281
precision score micro: 0.817193675889328
   recall score micro: 0.817193675889328
